# Kalman Filter

This notebook demonstrates the Kalman filter algorithm, which is used for estimating the state of a dynamic system from a series of noisy measurements. By visualizing what the Kalman filter models, we can get a clearer picture of how it works.

Let's say that we want to track the position of a bird in the sky. The bird largely moves in a straight line, but it is also a windy day, so the bird is knocked off course a little bit. We can assume that we have a specialized camera that detects the bird's position with a certain amount of noise. The Kalman filter will help us estimate the bird's position over time, even with the noisy measurements.

This notebook assumes you have already studied up on the basics of the Kalman filter presented [here](https://ajdillhoff.github.io/notes/tracking/#kalman-filters)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse
import matplotlib.transforms as transforms

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse
import matplotlib.transforms as transforms
from scipy.stats import chi2

def plot_covariance_ellipse(mean, cov, ax, n_std=3.0, facecolor='none', **kwargs):
    """
    Create a plot of the covariance confidence ellipse.
    
    Parameters
    ----------
    mean : array-like, shape (2,)
        The center point of the ellipse (x, y coordinates)
    cov : array-like, shape (2, 2)
        The covariance matrix
    ax : matplotlib.axes.Axes
        The axes object to draw the ellipse into
    n_std : float
        The number of standard deviations to determine the ellipse's radiuses
    facecolor : str
        The color to fill the ellipse
    **kwargs
        Forwarded to matplotlib.patches.Ellipse
    
    Returns
    -------
    matplotlib.patches.Ellipse
    """
    # Check inputs
    if len(mean) != 2 or cov.shape != (2, 2):
        raise ValueError("mean must be length 2, and cov must be 2x2")
    
    # Calculate eigenvalues and eigenvectors
    vals, vecs = np.linalg.eigh(cov)
    
    # Sort by eigenvalue in descending order
    order = vals.argsort()[::-1]
    vals = vals[order]
    vecs = vecs[:, order]
    
    # Angle of the first eigenvector
    angle = np.degrees(np.arctan2(vecs[1, 0], vecs[0, 0]))
    
    # Width and height are "full" widths, not radius
    width, height = 2 * n_std * np.sqrt(vals)
    
    # Create the ellipse
    ellipse = Ellipse(xy=mean, width=width, height=height, angle=angle, 
                      facecolor=facecolor, **kwargs)
    
    return ax.add_patch(ellipse)


# Set the random seed for reproducibility
np.random.seed(42)

n_measurements = 10
true_velocity = np.array([0.75, 0.5])  # True velocity of the bird
measurements = np.zeros((n_measurements, 4))  # Initial position of the bird

# Simulate the bird's movement
for i in range(1, n_measurements):
    # Update the position based on the true velocity
    measurements[i, :2] = measurements[i-1, :2] + true_velocity + np.random.normal(0, 0.3, size=2)  # Add some noise
    measurements[i, 2:] = measurements[i, :2] - measurements[i-1, :2]  # Relative position

# Plot the measurements and the true position of the bird
plt.figure(figsize=(10, 6))
plt.scatter(measurements[:, 0], measurements[:, 1])
plt.title('Bird Position Measurements')
plt.xlabel('X')
plt.ylabel('Y')
plt.grid()
plt.show()

# A basic Kalman filter

The Kalman filter is a recursive algorithm that estimates the state of a system over time. It consists of two main steps: prediction and update (correction).

The prediction step uses the current state estimate to predict the next state of the system. The update step uses the new measurement to correct the predicted state. The Kalman filter combines these two steps to produce an optimal estimate of the system's state.

For this simple tracking problem, our state will capture the position and velocity at each time step. That gives us a 4d state vector. We can predict the next state using a simple linear model following physics: 

\begin{align*}
\mathbf{p}_k &= \mathbf{p}_{k-1} + \Delta t \mathbf{v}_{k-1}\\
\mathbf{v}_k &= \mathbf{v}_{k-1}
\end{align*}

To simplify this as a matrix-vector operation, we can define the state transition matrix

$$
D = \begin{bmatrix}
1 & 0 & \Delta t & 0\\
0 & 1 & 0 & \Delta t\\
0 & 0 & 1 & 0\\
0 & 0 & 0 & 1
\end{bmatrix}
$$

At each time step, we compute two different state estimates. First, $\bar{\mathbf{x}}_i^{-}$ is the estimate of the state as predicted from the previous state. Second, $\bar{\mathbf{x}}_i^{+}$ is the estimate of the state after we have updated it with the new measurement.

For each new time step, we compute the predicted state $\bar{\mathbf{x}}_i^{-}$ using the state transition matrix $D$ and the previous state $\mathbf{x}_{i-1}$:

$$
\bar{\mathbf{x}}_i^{-} = D \bar{\mathbf{x}}_{i-1}^{+}
$$


# Tracking the first position

The first detection we get is the position of the bird at time $t=0$. From our tracker's perspective, we cannot determine which direction the bird is flying. In the absence of any additional information, we can only predict that the position at $t=1$ will be approximately close to the position at $t=0$.

To quantify this uncertainty, we will use a Gaussian distribution centered at the observed location that has unit variance. The variance of the Gaussian distribution represents our uncertainty about the bird's position. A larger variance means we are less certain about the position, while a smaller variance means we are more certain.

In the cell below, the state at $t=0$ is visualized. Additionally, the next measurement from $t=1$ is shown.

In [ ]:
state_position = np.zeros((n_measurements, 4))
state_position[0] = measurements[0]

state_covariance = np.zeros((n_measurements, 4, 4))
for i in range(n_measurements):
    state_covariance[i] = np.eye(4)

# Plot the first state
plt.figure(figsize=(10, 6))
plt.plot(state_position[0, 0], state_position[0, 1], 'o', label='State for $t=0$')
plt.plot(measurements[1, 0], measurements[1, 1], 'o', label='Observation for $t=1$')
plot_covariance_ellipse(state_position[0, :2], state_covariance[0, :2, :2], plt.gca(), n_std=1.0, edgecolor='red', facecolor='red', alpha=0.2)
plt.title('State Visualization')
plt.xlabel('Time')
plt.ylabel('State')
plt.xlim(-1, 2)
plt.ylim(-1, 2)
plt.legend()
plt.grid()
plt.gca().set_aspect('equal', adjustable='box')
plt.show()

# Computing the observation corrected state

Given the new observation at $t+1$, how do we correct our predicted state? The goal here is to reconcile the uncertainty of our predicted state with the uncertainty of the measurement. That is, we want to know the distribution over the union of these two distributions. This is achieved by multiplying the Gaussians together.

$$
\mathcal{N}(\bar{\mathbf{x}}_i^+, \Sigma_i^+) = \mathcal{N}(\bar{\mathbf{x}}_i^-, \Sigma_i^-) * \mathcal{N}(\mathbf{y}_i, \Sigma_{m_i})
$$

Solving for $\bar{\mathbf{x}}_i^+$ and $\Sigma_i^+$ yields

\begin{align*}
\bar{\mathbf{x}}_i^+ &= \bar{\mathbf{x}}_i^- + \mathcal{K}_i(\mathbf{y}_i - \bar{\mathbf{x}}_i^-)\\
\Sigma_i^+ &= \Sigma_i^- - \mathcal{K}_i \Sigma_i^-,
\end{align*}

where

$$
\mathcal{K}_i = \Sigma_i^-(\Sigma_i^- + \Sigma_{m_i})^{-1}.
$$

$\mathcal{K}_i$ is called the **Kalman gain**.



In [ ]:
# Update the state with the first measurement
observation_covariance = np.eye(4) * 0.5 # Assumed observation covariance -- could come from sensor noise
delta_t = 1 # Assumed time step between measurements

# State matrix
D = np.eye(4)
D[0, 2] = delta_t
D[1, 3] = delta_t

state_corrected = np.zeros((n_measurements, 4))
state_covariance_corrected = np.zeros((n_measurements, 4, 4))

# Since we are initializing the first state with the first measurement, our estimate for the next time step is the same.
# Remember, we haven't updated the state yet, so we are just copying the initial state.
# Note that this is a simplification. Typically, we would have at least 2 measurements when initializing the Kalman filter.
state_position[1] = state_position[0]
state_covariance[1] = state_covariance[0]

# Kalman filter update step for the first measurement
kalman_gain = state_covariance[1] @ np.linalg.inv(state_covariance[1] + observation_covariance)
state_corrected[1] += kalman_gain @ (measurements[1] - state_position[1])
state_covariance_corrected[1] = state_covariance[1] - kalman_gain @ state_covariance[1]

# Plot the corrected state
plt.figure(figsize=(10, 6))
plt.plot(state_position[0, 0], state_position[0, 1], 'o', label='State for $t=0$')
plt.plot(state_corrected[1, 0], state_corrected[1, 1], 'o', label='Corrected State for $t=1$')
plt.plot(measurements[1, 0], measurements[1, 1], 'o', label='Observation for $t=1$')
plot_covariance_ellipse(state_corrected[1, :2], state_covariance_corrected[1, :2, :2], plt.gca(), n_std=1.0, edgecolor='red', facecolor='red', alpha=0.2)
plt.title('Corrected State Visualization')
plt.xlabel('Time')
plt.ylabel('State')
plt.xlim(-1, 2)
plt.ylim(-1, 2)
plt.legend()
plt.grid()
plt.gca().set_aspect('equal', adjustable='box')
plt.show()


# Continuing on

The first prediction and update step is complete. The more information we have, the more accurate our estimate will be. The cell below shows the full result of tracking the bird's position through 10 time steps.

In [ ]:
# Update the state throughout the measurements
for i in range(2, n_measurements):
    # Predict the next state
    state_position[i] = D @ state_corrected[i-1]
    state_covariance[i] = D @ state_covariance_corrected[i-1] @ D.T + observation_covariance

    # Kalman filter update step
    kalman_gain = state_covariance[i] @ np.linalg.inv(state_covariance[i] + observation_covariance)
    state_corrected[i] = state_position[i] + kalman_gain @ (measurements[i] - state_position[i])
    state_covariance_corrected[i] = state_covariance[i] - kalman_gain @ state_covariance[i]

# Print final covariance
print("Final state covariance:")
print(state_covariance_corrected[-1])

# Plot the corrected states
plt.figure(figsize=(10, 6))
plt.plot(state_position[:, 0], state_position[:, 1], 'o', label='Predicted State')
plt.plot(state_corrected[:, 0], state_corrected[:, 1], 'o', label='Corrected State')
plt.plot(measurements[:, 0], measurements[:, 1], 'o', label='Observations')
for i in range(n_measurements):
    plot_covariance_ellipse(state_corrected[i, :2], state_covariance_corrected[i, :2, :2], plt.gca(), n_std=1.0, edgecolor='red', facecolor='red', alpha=0.2)
plt.title('Kalman Filter State Visualization')
plt.xlabel('X')
plt.ylabel('Y')
plt.legend()
plt.grid()
plt.gca().set_aspect('equal', adjustable='box')
plt.show()
